In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model
import sklearn.metrics as sklm
from time import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [2]:
t0 = time()
data = pd.read_csv('train_values_prepped.csv')
labels = pd.read_csv('train_labels_prepped.csv')

In [3]:
data['rate_spread'] = labels['rate_spread'] #Adding rate spread to data
data = data.loc[data['rate_spread'] < 9]
print(data.isnull().sum())

row_id                 0
loan_type              0
property_type          0
loan_purpose           0
occupancy              0
loan_amount            0
preapproval            0
msa_md                 0
state_code             0
county_code            0
applicant_ethnicity    0
applicant_race         0
applicant_sex          0
lender                 0
co_applicant           0
rate_spread            0
dtype: int64


In [4]:
print(data.shape)
scaler = MinMaxScaler()
temp = data['loan_amount']
temp =np.array(temp)
temp = temp.reshape(-1,1)
scaler.fit(temp)
data['loan_amount'] = scaler.transform(temp)


(191426, 16)


In [5]:
data['property_type'][0] = 3


cat = ['property_type','loan_purpose','occupancy','preapproval','applicant_ethnicity','applicant_race','applicant_sex',
      'co_applicant']

def encode_string(cat_feature):
    ## First encode the strings to numeric categories
    enc = preprocessing.LabelEncoder()
    enc.fit(cat_feature)
    enc_cat_feature = enc.transform(cat_feature)
    ## Now, apply one hot encoding
    ohe = preprocessing.OneHotEncoder()
    encoded = ohe.fit(enc_cat_feature.reshape(-1,1))
    return encoded.transform(enc_cat_feature.reshape(-1,1)).toarray()
    
cat_features = encode_string(data['loan_type'])
for col in cat:
    temp = encode_string(data[col])
    cat_features = np.concatenate([cat_features, temp], axis = 1)
    print(str(col) + " " + str(temp.shape))

data['property_type'][0] = 1

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


property_type (191426, 3)
loan_purpose (191426, 3)
occupancy (191426, 3)
preapproval (191426, 3)
applicant_ethnicity (191426, 4)
applicant_race (191426, 7)
applicant_sex (191426, 4)
co_applicant (191426, 2)


/home/nbuser/anaconda3_420/lib/python3.5/site-packages/ipykernel/__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
data['Rate_Higher_Than_2'] = data['rate_spread'].apply(lambda x: 1.0 if x > 2 else 0.0)

In [7]:
#Concats numerical scaled features and cat_features. Then uses kbest to select the best 16
temp = data['loan_amount']
temp = np.array(temp)
temp = temp.reshape(-1,1)

all_features = np.concatenate([cat_features, temp], axis = 1)

y = data['Rate_Higher_Than_2']

bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(all_features, y)
dfscores = pd.DataFrame(fit.scores_)
# dfcolumns = pd.DataFrame(numerical)
featureScores = pd.concat([dfscores], axis=1)
featureScores.columns = ['Score']
featureScores = featureScores.round(3)
print(featureScores)
print(featureScores.nlargest(15,'Score'))  #print 10 best features
Features_reduced = fit.transform(all_features)
print(Features_reduced.shape)

        Score
0   22415.707
1   17618.409
2      51.426
3     449.372
4    6831.935
5   35901.735
6      84.890
7     702.932
8    9849.688
9       5.202
10      6.121
11     89.893
12     15.657
13   1182.698
14   5785.330
15   2330.158
16    949.359
17     69.238
18   5100.127
19     38.515
20    171.116
21     94.380
22    173.384
23     27.046
24      4.097
25    674.731
26     35.454
27      0.074
28     24.490
29    210.694
30      8.164
31    308.995
32    510.377
33   3843.891
        Score
5   35901.735
0   22415.707
1   17618.409
8    9849.688
4    6831.935
14   5785.330
18   5100.127
33   3843.891
15   2330.158
13   1182.698
16    949.359
7     702.932
25    674.731
32    510.377
3     449.372
(191426, 10)


In [8]:
Features_reduced = fit.transform(all_features)
print(Features_reduced.shape)

(191426, 10)


In [9]:
import numpy.random as nr

nr.seed(9988)
labels = data['rate_spread']

indx = range(Features_reduced.shape[0])
indx = ms.train_test_split(indx, test_size = 50000)
x_train = Features_reduced[indx[0],:]
y_train = np.ravel(labels[indx[0]])
x_test = Features_reduced[indx[1],:]
y_test = np.ravel(labels[indx[1]])

In [10]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values = np.nan, strategy = 'most_frequent')
y_train = y_train.reshape(-1,1)
imp = imp.fit(y_train)
y_train = imp.transform(y_train)

imp2 = Imputer(missing_values = np.nan, strategy = 'most_frequent')
y_test = y_test.reshape(-1,1)
imp2 = imp2.fit(y_test)
y_test = imp2.transform(y_test)
print(np.isnan(y_train).sum())
print(np.isnan(x_train).sum())
print(np.isnan(y_test).sum())
print(np.isnan(x_test).sum())



0
0
0
0


In [12]:
x_train = x_train[:int(len(x_train)/100)]
y_train = y_train[:int(len(y_train)/100)]

In [21]:
from sklearn import svm
clf = svm.SVC()
clf.fit(x_train,y_train)
probabilities = clf.predict(x_test)
print(probabilities.shape)

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(50000,)


In [22]:
def score_model(probs, threshold):
    return np.array([1 if x > threshold else 0 for x in probs[:,1]])
scores = score_model(probabilities, 0.5)
print(np.array(scores[:15]))
print(y_test[:15])

IndexError: too many indices for array